In [ ]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

from torch.utils.data import Dataset, DataLoader, BatchSampler, random_split
from torchvision import transforms
from PIL import Image

In [ ]:
# Train only subclass classification for a given superclass
superclass_idx = 0 # 0 for dog, 1 for bird, 2 for reptile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/NNDL_Project

Mounted at /content/drive
/content/drive/MyDrive/NNDL_Project


In [ ]:
%%capture
! unzip -o train_shuffle.zip

In [ ]:
%%capture
! unzip -o test_shuffle.zip

In [ ]:
# Create Dataset class for multilabel classification
class MultiClassImageDataset(Dataset):
    def __init__(self, ann_df, sub_map_df, img_dir, transform=None):
        self.ann_df = ann_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.ann_df)

    def __getitem__(self, idx):
        img_name = self.ann_df['image'][idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        sub_idx = self.ann_df['subclass_index'][idx]
        sub_label = self.sub_map_df['class'][sub_idx]

        if self.transform:
            image = self.transform(image)

        return image, sub_idx, sub_label

class MultiClassImageTestDataset(Dataset):
    def __init__(self, sub_map_df, img_dir, transform=None):
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self): # Count files in img_dir
        return len([fname for fname in os.listdir(self.img_dir)])

    def __getitem__(self, idx):
        img_name = str(idx) + '.jpg'
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name

In [ ]:
train_ann_df = pd.read_csv('train_data.csv')
train_ann_df = train_ann_df[train_ann_df['superclass_index'] == superclass_idx]
train_ann_df = train_ann_df.reset_index(drop=True)

super_map_df = pd.read_csv('superclass_mapping.csv')
sub_map_df = pd.read_csv('subclass_mapping.csv')

train_img_dir = 'train_shuffle'
test_img_dir = 'test_shuffle'

image_preprocessing = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0), std=(1)),
])

# Create train and val split
train_dataset = MultiClassImageDataset(train_ann_df, sub_map_df, train_img_dir, transform=image_preprocessing)
train_dataset, val_dataset = random_split(train_dataset, [0.9, 0.1])

# Create test dataset
test_dataset = MultiClassImageTestDataset(sub_map_df, test_img_dir, transform=image_preprocessing)

# Create dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True)

test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         shuffle=False)

In [ ]:
print(train_ann_df['superclass_index'].value_counts())
train_ann_df.head()

0    1850
Name: superclass_index, dtype: int64


,image,superclass_index,subclass_index
0,1.jpg,0,42
1,4.jpg,0,4
2,9.jpg,0,41
3,12.jpg,0,20
4,13.jpg,0,19


In [ ]:
train_ann_df[train_ann_df['superclass_index'] == superclass_idx]

,image,superclass_index,subclass_index
0,1.jpg,0,42
1,4.jpg,0,4
2,9.jpg,0,41
3,12.jpg,0,20
4,13.jpg,0,19
...,...,...,...
1845,6311.jpg,0,4
1846,6313.jpg,0,60
1847,6314.jpg,0,27
1848,6319.jpg,0,86


In [ ]:
super_map_df

,index,class
0,0,bird
1,1,dog
2,2,reptile
3,3,novel


In [ ]:
sub_map_df.head()

,index,class
0,0,"Scotch terrier, Scottish terrier, Scottie"
1,1,"African chameleon, Chamaeleo chamaeleon"
2,2,standard schnauzer
3,3,terrapin
4,4,"great grey owl, great gray owl, Strix nebulosa"


In [ ]:
sub_map_df[sub_map_df['class']=='novel']

,index,class
87,87,novel


In [ ]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.block1 = nn.Sequential(
                        nn.Conv2d(3, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.Conv2d(32, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.Conv2d(32, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.MaxPool2d(2, 2)
                      )

        self.block2 = nn.Sequential(
                        nn.Conv2d(32, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.Conv2d(64, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.Conv2d(64, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.MaxPool2d(2, 2)
                      )

        self.block3 = nn.Sequential(
                        nn.Conv2d(64, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.Conv2d(128, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.Conv2d(128, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.MaxPool2d(2, 2)
                      )

        self.fc1 = nn.Linear(4*4*128, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3b = nn.Linear(128, 88)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        sub_out = self.fc3b(x)
        return sub_out

class Trainer():
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, test_loader=None, device='cuda'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device

    def train_epoch(self):
        running_loss = 0.0
        device = self.device
        for i, data in enumerate(self.train_loader):
            inputs, sub_labels = data[0].to(device), data[1].to(device)

            self.optimizer.zero_grad()
            sub_outputs = self.model(inputs)
            loss = self.criterion(sub_outputs, sub_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Training loss: {running_loss/i:.3f}')

    def validate_epoch(self):
        sub_correct = 0
        total = 0
        running_loss = 0.0
        device = self.device
        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                inputs, sub_labels = data[0].to(device), data[1].to(device)

                sub_outputs = self.model(inputs)
                loss = self.criterion(sub_outputs, sub_labels)
                _, sub_predicted = torch.max(sub_outputs.data, 1)

                total += sub_labels.size(0)
                sub_correct += (sub_predicted == sub_labels).sum().item()
                running_loss += loss.item()

        print(f'Validation loss: {running_loss/i:.3f}')
        print(f'Validation subclass acc: {100 * sub_correct / total:.2f} %')

    def test(self, save_to_csv=False, return_predictions=False):
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')

        # Evaluate on test set, in this simple demo no special care is taken for novel/unseen classes
        test_predictions = {'image': [], 'subclass_index': []}
        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                inputs, img_name = data[0].to(device), data[1]

                sub_outputs = self.model(inputs)
                _, sub_predicted = torch.max(sub_outputs.data, 1)

                test_predictions['image'].append(img_name[0])
                test_predictions['subclass_index'].append(sub_predicted.item())

        test_predictions = pd.DataFrame(data=test_predictions)

        if save_to_csv:
            test_predictions.to_csv(f'test_predictions_subclass_superclass{superclass_idx}.csv', index=False)

        if return_predictions:
            return test_predictions

In [ ]:
# Init model and trainer
device = 'cuda'
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
trainer = Trainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

In [ ]:
# Training loop
for epoch in range(20):
    print(f'Epoch {epoch+1}')
    trainer.train_epoch()
    trainer.validate_epoch()
    print('')

print('Finished Training')

Epoch 1
Training loss: 2.046
Validation loss: 2.999
Validation subclass acc: 41.08 %

Epoch 2
Training loss: 1.772
Validation loss: 2.471
Validation subclass acc: 44.86 %

Epoch 3
Training loss: 1.584
Validation loss: 2.187
Validation subclass acc: 50.81 %

Epoch 4
Training loss: 1.555
Validation loss: 2.156
Validation subclass acc: 51.89 %

Epoch 5
Training loss: 1.305
Validation loss: 2.253
Validation subclass acc: 52.43 %

Epoch 6
Training loss: 1.269
Validation loss: 2.238
Validation subclass acc: 50.27 %

Epoch 7
Training loss: 0.940
Validation loss: 2.244
Validation subclass acc: 54.05 %

Epoch 8
Training loss: 0.983
Validation loss: 1.957
Validation subclass acc: 60.54 %

Epoch 9
Training loss: 1.012
Validation loss: 2.188
Validation subclass acc: 55.14 %

Epoch 10
Training loss: 0.844
Validation loss: 2.262
Validation subclass acc: 56.76 %

Epoch 11
Training loss: 0.831
Validation loss: 1.909
Validation subclass acc: 59.46 %

Epoch 12
Training loss: 0.784
Validation loss: 1.871

In [ ]:
trainer.test(save_to_csv=False, return_predictions=True)

'''
This simple baseline scores the following test accuracy

Superclass Accuracy
Overall: 43.83 %
Seen: 61.11 %
Unseen: 0.00 %

Subclass Accuracy
Overall: 2.03 %
Seen: 9.56 %
Unseen: 0.00 %
'''

'\nThis simple baseline scores the following test accuracy\n\nSuperclass Accuracy\nOverall: 43.83 %\nSeen: 61.11 %\nUnseen: 0.00 %\n\nSubclass Accuracy\nOverall: 2.03 %\nSeen: 9.56 %\nUnseen: 0.00 %\n'